## Electricity consumption and price analysis

In [25]:
import plotly.graph_objects as go
import pandas as pd

In [26]:
df = pd.read_hdf('data/h5/cleaned_data.h5')
df

,TINKLAS,OBT_PAVADINIMAS,OBJ_GV_TIPAS,OBJ_NUMERIS,P+,PL_T,P-
2,Alytaus regiono tinklas,Butas,Ne GV,37502,0.0500,2020-06-30 00:00:00,0.0
5,Vilniaus regiono tinklas,Butas,Ne GV,600022,0.0230,2020-06-30 00:00:00,0.0
13,Šiaulių regiono tinklas,Namas,G,859552,0.4135,2020-06-30 00:00:00,0.0
17,Kauno regiono tinklas,Butas,N,197974,0.1058,2020-06-30 00:00:00,0.0
23,Vilniaus regiono tinklas,Butas,Ne GV,535458,0.0400,2020-06-30 00:00:00,0.0
...,...,...,...,...,...,...,...
11816434,Vilniaus regiono tinklas,Butas,Ne GV,585738,0.0260,2022-05-01 01:00:00,0.0
11816450,Vilniaus regiono tinklas,Butas,Ne GV,578178,0.0940,2022-05-01 01:00:00,0.0
11816458,Vilniaus regiono tinklas,Butas,Ne GV,575298,0.0380,2022-05-01 01:00:00,0.0
11816461,Vilniaus regiono tinklas,Butas,Ne GV,573378,0.0290,2022-05-01 01:00:00,0.0


### A. How much electricity (kwh) did apartment 771822 (object number) consume in 2 years?

In [27]:
consumption = df[df['OBJ_NUMERIS'] == 771822].copy()

total_kwh =  df[df['OBJ_NUMERIS'] == 771822]['P+'].sum()

print(f"Total electricity consumption for apartment 771822 over 2 years: {total_kwh:.2f} kWh")

Total electricity consumption for apartment 771822 over 2 years: 3046.78 kWh


### B. How much electricity (kWh) is consumed on average (median) per day by one apartment and one house in each month of the year.


In [67]:
df['PL_T'] = pd.to_datetime(df['PL_T'])
df['Date'] = df['PL_T'].dt.date

daily_consumptions = df.groupby(['OBJ_NUMERIS', 'OBT_PAVADINIMAS','Date'])['P+'].sum().reset_index()

daily_consumptions


,OBJ_NUMERIS,OBT_PAVADINIMAS,Date,P+
0,37082,Butas,2020-06-01,7.1160
1,37082,Butas,2020-06-02,10.2460
2,37082,Butas,2020-06-03,7.3130
3,37082,Butas,2020-06-04,5.2590
4,37082,Butas,2020-06-05,10.5530
...,...,...,...,...
77375,862312,Namas,2022-05-27,6.8294
77376,862312,Namas,2022-05-28,6.5415
77377,862312,Namas,2022-05-29,7.8268
77378,862312,Namas,2022-05-30,10.7717


In [73]:
# Removing hours from dates, while grouping by date summing hours 
df['date_without_hours'] = df['PL_T'].dt.date
df['month'] = df['PL_T'].dt.month

result = (df.groupby(['OBJ_NUMERIS', 'OBT_PAVADINIMAS', 'month', 'date_without_hours'])
          .agg({'P+': 'sum'})                                  
          .groupby(['OBT_PAVADINIMAS', 'month'])
          .agg({'P+': 'median'})                                 
          .reset_index()
          .pivot(index='month', columns='OBT_PAVADINIMAS', values='P+'))

result


OBT_PAVADINIMAS,Butas,Namas
month,,
1,4.1100,20.98355
2,4.1220,19.13935
3,3.8865,15.75300
4,3.6630,12.52200
5,3.5920,8.94125
6,2.7534,6.38740
7,2.5775,6.31660
8,2.7495,7.14265
9,3.3500,9.58200


In [74]:
def visualize_electricity_consumption(df):
    
    fig_line = go.Figure()

    # Add line traces for apartments and houses
    fig_line.add_trace(
        go.Scatter(x=result.index, 
                  y=result['Butas'],
                  name='Apartment',
                  line=dict(color='#636EFA', width=2),
                  mode='markers')
    )

    fig_line.add_trace(
        go.Scatter(x=result.index, 
                  y=result['Namas'],
                  name='House',
                  line=dict(color='#EF553B', width=2),
                  mode='markers')
    )

    # Update line plot layout
    fig_line.update_layout(
        title='Daily Electricity Consumption by Property Type',
        title_x=0.5,
        xaxis_title='Month',
        yaxis_title='Consumption (kWh)',
        height=500,
        showlegend=True,
        legend=dict(
            yanchor="bottom",
            y=1.02,
            xanchor="center",
            x=0.5,
            orientation="h"
        )
    )

    # Update x-axis
    fig_line.update_xaxes(
        ticktext=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
        tickvals=list(range(1, 13))
    )
       
    fig_line.show()
    fig_line.write_image(
    "images/Daily_electricity_consumption_by_property_type.png",
    scale=3,         
    width=1200,      
    height=800       
)

In [75]:
visualize_electricity_consumption(df)

In [76]:
def create_table(result):
    # Define month names
    month_names = ['January', 'February', 'March', 'April', 'May', 'June',
                  'July', 'August', 'September', 'October', 'November', 'December']
    
    # Create table data
    table_data = [
        ['Month', 'Apartment (kWh)', 'House (kWh)']  # Header
    ]
    
    # # Add data rows
    for month in range(1, 13):
        if month in result.index:
            apt_value = result.loc[month, 'Butas']
            house_value = result.loc[month, 'Namas']
            table_data.append([
                month_names[month-1],
                f"{apt_value:.3f}",
                f"{house_value:.3f}"
            ])

    # Create figure for table
    fig_table = go.Figure(data=[go.Table(
        header=dict(
            values=table_data[0],
            fill_color='#C8D4E3',
            align='center',
            font=dict(size=11, color='black'),
            height=25
        ),
        cells=dict(
            values=list(zip(*table_data[1:])),
            fill_color='white',
            align=['center', 'center', 'center'],
            font=dict(size=10),
            height=20
        )
    )])


    fig_table.update_layout(
        title='Average electricity consumption per day (kWh)',
        title_x=0.5,
        height=325,
        margin=dict(l=20, r=20, t=40, b=20)
    )

    fig_table.show()
    
    fig_table.write_image(
    "images/Average_electricity_consumption.png",
    scale=3,         
    width=1200,      
    height=350       
)

In [77]:
create_table(result)


### C. When was the most expensive electricity in 2022 (date and time) and how much is this price as a percentage of average electricity prices for such an hour in that year?

##### Data on the NordPool exchange: https://www.litgrid.eu/index.php/sistemos-duomenys/elektros-energijoskainos/86.

In [33]:
df_prices = pd.read_hdf('data/h5/electricity_prices.h5', key='prices')

df_prices['date'] = pd.to_datetime(df_prices['date'])

# Maximum price and its date/time
max_price_row = df_prices.loc[df_prices['price'].idxmax()]
max_price = max_price_row['price']
max_price_datetime = max_price_row['date']
max_price_hour = max_price_datetime.hour


avg_hour_price = df_prices[df_prices['date'].dt.hour == max_price_hour]['price'].mean()

percentage = (max_price / avg_hour_price) * 100

print(f"2022 Nordpool price analysis:")
print(f"Date and time: {max_price_datetime}")
print(f"Price: {max_price:.2f} EUR/MWh")
print(f"Average price for hour {max_price_hour:02d}:00: {avg_hour_price:.2f} EUR/MWh")
print(f"The highest price is {percentage:.1f}% of the average price for that hour")

2022 Nordpool price analysis:
Date and time: 2022-08-17 18:00:00
Price: 4000.00 EUR/MWh
Average price for hour 18:00: 278.98 EUR/MWh
The highest price is 1433.8% of the average price for that hour
